# Tallberg deposit: Exploration practical
## Reconnaissance: Till Geochemistry

You are conducting a reconnaissance study of the **Jörn granite** to explore for porphyry copper deposits. You have collected till samples from the locations on the map below ana analysed them for copper contents listed in the data table below, courtesy of SGU. ( We expect you to plot 15-20 points with anomalous concentrations ). The ice-flow direction is indicated by symbol D i.e. : <img src=https://raw.githubusercontent.com/kiarash13p/Mineral-Resource-Geology-and-Modelling/main/Images/D%20arrow.png width=80 height=40 display='inline' style="vertical-align:top">


<img src=https://raw.githubusercontent.com/kiarash13p/Mineral-Resource-Geology-and-Modelling/main/Images/map.png width=700 height=600 style="display: block; margin: 0 auto">

### Geological Setting:

The Tallberg porphyry deposit is found amongst the **Jörn granite complex** , adjacent to the _Skellefte_ region, Northern Sweden.

<img src=https://raw.githubusercontent.com/kiarash13p/Mineral-Resource-Geology-and-Modelling/main/Images/Gelogical%20Setting.png width=600 height=500 style="display: block; margin: 0 auto">

### Lithology:

The **Jörn granite complex** (JGC) is a multiple intrusion with three, or possibly four, intrusive phases. The batholith is zoned, with an older, least differentiated intrusive phases GI of  _*(Wilson et al. 1987)*_ forming an outer zone (Fig. 2). A smaller and younger granitoid intrusion occurs around the village of Jörn (GII), and a large central granite diapir (GIII) cuts the GI and GII phases. _*(Wilson et al. 1987)*_ also recognize a GIV phase in the central Gill diapir.

<div style="display: flex;justify-content: center">
<img src=https://raw.githubusercontent.com/kiarash13p/Mineral-Resource-Geology-and-Modelling/main/Images/Geological%20Setting%202.png width=500 height=500 style="margin: 10px 35px">
<img src=https://raw.githubusercontent.com/kiarash13p/Mineral-Resource-Geology-and-Modelling/main/Images/Geological%20Setting%203.png width=500 height=500 style="margin: 10px 35px">
</div>

Before jumping to first question, a little preparation is needed:

### Installing required packages

Following command cell, some of the libraries neeeded most for this assignment has been mentioned. If you come up with new ideas in which new libraries required, feel free to add them here.
It is recommended to unify all `import`s on one cell for better readability and explainability of the code.

If you are not comfortable with installing packages here ( or you want to install them on a new virtual environment ), execute `pip` command like:

```bash
pip install datasets
```

Notice that for installing packages on your virtual environment, you need to activate it beforehand or modify `pip` command parameters! Visit [**Documentation**](https://docs.python.org/3/library/venv.html) for more information.

In [ ]:
%pip install -q --upgrade pip
%pip install -q datasets
%pip install -q matplotlib
%pip install -q numpy
%pip install -q scipy

In [ ]:
from datasets import load_dataset , Value
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.ticker import PercentFormatter
import numpy as np
import scipy
from scipy import stats

#### Color maps
As an engineer, you have to learn how to present or depict your achievements and discoveries beautifully. Hereby, we are initializing a color map, in which intervals and their associated color setting has been defined.

Following codes are just examples for better demonstration. If you come up with new color designing or other approaches rather than `matplotlib`, feel free to change the code!

In [ ]:
plt.style.use('seaborn-v0_8')
# change the style to one of these values if you want
plt.style.available

In [ ]:
viridis        =   mpl.cm.viridis
viridis

In [ ]:
newcmap         =   mpl.colors.ListedColormap( viridis(np.linspace(0,3,128)) , name='Copper (PPM)' )
newcmap

In [ ]:
norm            =   mpl.colors.Normalize( vmin=0.0 , vmax= 32 ) # you can change maximum later
fig_colorbar , ax_colorbar      =   plt.subplots( figsize = (5,1) , layout='constrained' )
fig_colorbar.colorbar(
    mpl.cm.ScalarMappable( norm = norm , cmap = viridis ),
    orientation='horizontal',
    cax=ax_colorbar,
)

In [ ]:
bounds       =   [ 1.0 , 2.5 , 4 , 7 , 10 , 15 ] # you change these values based on your needs later
bnorm        =   mpl.colors.BoundaryNorm( bounds , viridis.N , extend='both')
fig_colorbar , ax_colorbar      =   plt.subplots( figsize = (5,1) , layout='constrained' )
fig_colorbar.colorbar(
    mpl.cm.ScalarMappable( norm = bnorm , cmap = viridis ),
    orientation='horizontal',
    cax=ax_colorbar,
)

### Importing data

Data required for this assignment has been uploaded on Huggingface ( known as 🤗 ) Datasets, which is lightweight library providing many facilities such as enabling dataloaders and efficient data preprocessing. Visit [🎓 **Documentation**](https://huggingface.co/docs/datasets/) for more information if you are interested!

`kiarash13p` is my username and `Tallberg` is the name of the dataset we are looking to investigate


In [ ]:
Cu_dataset      =   load_dataset( 'kiarash13p/Tallberg')['train']
Cu_dataset.features

### Data Conversion
In case of facing issues on **32-bit operating systems** or due to other reasons, you might be required to execute the following codes to cast conversion on columns
```python
new_features            =   Cu_dataset.features.copy()
new_features['X']       =   Value( dtype='int32' , id=None )
new_features['Y']       =   Value( dtype='int32' , id=None )
new_features['Cu_ppm']  =   Value( dtype='float32' , id=None ) # or float, but not use double ( which is float64 )
Cu_dataset.cast( new_features )
```

### Knowing dataset
By considering that data is cleaned of any missing or unsual values, lets see what is happening inside the dataset

In [ ]:
fig , ((ax1,ax2),(ax3,ax4))    =   plt.subplots( nrows= 2 , ncols= 2 , figsize = (12,7) , tight_layout = True  )
binsize     =   45
fig.suptitle( "Primary plots of copper distribution ( in PPM )" )
ax1_n , ax1_bins , ax1_patches          =   ax1.hist( Cu_dataset['Cu_ppm'] , bins = binsize )
ax1.set_title("Histogram plot")
ax1.set_xlabel("Cu")
ax1.set_ylabel("Frequency")
# Coloring by height of the bins
fracs       =   ax1_n / ax1_n.max()
ax1_norm    =   mpl.colors.Normalize( fracs.min() , fracs.max() )
for _frac , _patch in zip( fracs , ax1_patches ):
    _patch.set_facecolor( viridis( ax1_norm( _frac )))

ax2_n , ax2_bins , ax2_patches          =   ax2.hist( Cu_dataset['Cu_ppm'] , bins = binsize , density=True , color=viridis(0.5) , alpha = 0.3 )
ax2.set_title("Density plot")
ax2.set_xlabel("Cu")
ax2.set_ylabel("Percent")
ax2.yaxis.set_major_formatter( PercentFormatter(xmax=1))

ax3_n , ax3_bins , ax3_patches          =   ax3.hist( Cu_dataset['Cu_ppm'] , bins = binsize , cumulative=True , color=viridis(0.7) , histtype='step' )
ax3.set_title("Cumulative plot")
ax3.set_xlabel("Cu")
ax3.set_ylabel("Number of samples")
stats.probplot( Cu_dataset['Cu_ppm'] , plot=ax4 )

plt.show()

***
## Question 1

Predict giving reasons, where to focus further exploration for porphyry copper deposit.

Notes:
-   You are required to provide proofs for your claims and thoughts
-   Feel free to depict more charts and diagrams to increase your understanding about the dataset
***

***
## Question 2

Plot the Tallberg deposit at X=1693000 and Y=7221000. How does this compare to the prediction you made from the till geochemistry?

Notes:
-   You are required to provide proofs for your claims and thoughts
-   Feel free to depict more charts and diagrams to increase your understanding about the dataset
-   You are required to plot copper distribution in contours given the X and Y available on dataset
***

### References

_*Wilson, M.R., Sehlstedt, S., Claesson, L.Å., Smellie, J.A., Aftalion, M., Hamilton, P.J. and Fallick, A.E., 1987. Jörn: an early Proterozoic intrusive complex in a volcanic-arc environment, north Sweden. Precambrian Research, 36(3-4), pp.201-225*_ ([link](https://www.sciencedirect.com/science/article/pii/0301926887900210))